In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt

# Filtro de Kalman

## Introducción

El filtro de Kalman es un estimador lineal ampliamente utilizado perteneciente a la familia de los filtros Gaussianos. El objetivo del filtro del Kalman es estimar el estado interno de un sistema lineal a partir de observaciones ruidosas que llegan secuencialmente. El filtro de Kalman actualiza su estado en base a estas observaciones. Recibe el nombre Rudolf Kálmán  (May 19, 1930 – July 2, 2016), quien fue uno de los desarrolladores de esta teoría. Tal vez uno de los usos más icónicos del filtro de Kalman fue para estimar la trayectoria de los cohetes del programa Apolo.

## Formulación

El estado del sistema es una variable latente que denominaremos $x$. Nos interesa estimar esta variable en los instantes $k=1,\ldots,n$. 

Uno de los supuestos del filtro de Kalman es que el sistema dinámico que estamos estimando es lineal y que se puede escribir como

$$
x_k = F x_{k-1} + w_{k-1},
$$

donde $F$ es la matriz de transición de estado y $w$ es una variable aleatoria que refleja el ruido del proceso y que

$$
w_{k-1} \sim \mathcal{N}(0, Q),
$$

es decir que se distribuye Gaussiano con media zero y covarianza $Q$.

También tenemos un sensor que nos entrega observaciones $z$ que asumimos se relacionan con el estado de la siguiente forma

$$
z_k = H x_k + \nu_k,
$$

donde $H$ es la matriz de observación y $\nu$ es una variable aleatoria que refleja el ruido del sensor y que 

$$
v \sim \mathcal{N}(0, R),
$$

es decir que el ruido del sensor es gaussiano con media cero y matriz de covarianza $R$.

El objetivo es estimar iterativamente $x_k$ dado $R$, $H$, $F$, $Q$, $x_0$ y las mediciones $z_k$



## Algoritmo

El filtro de Kalman tiene dos etapas: la etapa de predicción y la etapa de actualización o corrección.

En lo que sigue se utiliza la notación $\hat x$ para indicar una estimación de $x$. También se utilizarán superíndices $+$ y $-$ para denotar información a posteriori y a priori, respectivamente

En la etapa de predicción tenemos que

$$
\hat x_k^{-} = F \hat x_{k-1}^{+} 
$$

y su covarianza es

$$
P_k^{-} = F P_{k-1}^{+} F^T + Q
$$

Luego en la etapa de actualización primero medimos el error de nuestra predicción o residuo

$$
r_k = z_k - H \hat x_k^{-}
$$

y calculamos la ganacia del filtro

$$
K = \frac{P_k^{-} H^T}{R + H P_k^{-} H^T}
$$

con lo que podemos actualizar el estado

$$
\hat x_k^{+} = \hat x_k^{-} + K r
$$

y su covarianza

$$
P_k^{-} = (I - K H) P_{k}^{-}
$$

Observando la ganacia del filtro podemos notar que si  

- Si $R$ es mucho más grande que $P_k^{-}$ entonces $K$ tiende a cero y $\hat x_k^{+} \approx \hat x_k^{-}$. Es decir que si el sensor es muy ruidoso confiamos más en nuestra estimación
- Si $R$ tiende a cero entonces $K$ tiende a la unidad y $\hat x_k^{+}$ será modificado de forma importante por $z_k$


## Ejemplo

Digamos que tenemos un cohete que viaja en una dimensión. Nos interesa estimar la posición, velocidad y aceleración del cohete en todo instante. El cohete tiene un sensor que nos permite medir la aceleración. 

Recordemos que podemos escribir la velocidad como

$$
v_k = v_{k-1} + a_{k-1} \Delta t
$$

y la posición como 

$$
p_k = p_{k-1} + v_{k-1} \Delta t  + \frac{1}{2} a_{k-1} \Delta t^2
$$

Luego podemos escribir el vector de estado como

$$
x_k = \begin{pmatrix} p_k \\ v_k \\ a_k \end{pmatrix} = \begin{pmatrix} 1 & \Delta t & \frac{1}{2} \Delta t^2\\ 0 & 1 & \Delta t \\ 0 & 0 & 1\end{pmatrix} \begin{pmatrix} p_{k-1} \\ v_{k-1} \\ a_{k-1}\end{pmatrix} + w_k
$$

de donde podemos reconocer $F$. Dada la definición de estado y que sólo se observa la aceleración entonces tenemos que 

$$
z_k = \begin{pmatrix} 0 & 0 & 1 \end{pmatrix} \begin{pmatrix} p_k \\ v_k \\ a_k \end{pmatrix} + \nu_k
$$

De donde reconocemos $H$

Asumamos la posición inicial es cero,  la velocidad inicial es 5 y que la aceleración inicial es cero. Asumiremos también que $\Delta t = 0.01$

In [ ]:
class KalmanFilter1D:
    def __init__(self, x0, P0, F, H, R, Q):
        self.x = x0
        self.P = P0
        self.R = R
        self.Q = Q
        self.F = F
        self.B = B
        self.H = H

    def predict(self):
        self.x = np.dot(self.F, self.x)
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x, self.P
        
    def update(self, z):
        r =  z - np.dot(self.H, self.x)
        pi = np.dot(self.P, self.H.T)
        K = pi / (np.dot(self.H, pi) + self.R)
        self.x += np.dot(K, r)
        self.P -= np.dot(np.dot(K, self.H), self.P)

dt = 0.01
x0 = np.array([0., 5., 0.])
F = np.array([[1, dt, 0.5*dt**2], [0, dt, 1], [0, 0, 1]])
H = np.array([0, 0, 1])

¿Cómo cambia el resultado con distintos valores de $R$ y $Q$?

In [ ]:
P0 = np.diag([1.0]*3)
R = 1e-1
Q = 1e-5

z_real = np.zeros(shape=(200))
z_real[50:] += 1
z = z_real + np.random.randn(len(z_real))*np.sqrt(R)

fig, ax = plt.subplots(2, tight_layout=True)
ax[0].plot(np.arange(len(z_real))*dt, z_real, label='Real')
ax[0].plot(np.arange(len(z_real))*dt, z, 'o', label='Observada')

kalman = KalmanFilter1D(x0, P0, F, H, R, Q)

x_pred = [x0]
for z_ in z:
    x, P = kalman.predict()
    kalman.update(z_)    
    x_pred.append(x)

x_pred = np.stack(x_pred)
ax[0].plot(np.arange(len(z_real)+1)*dt, x_pred[:, -1], label='Estimada')
ax[0].set_ylabel('Aceleración')
ax[0].legend()
ax[1].plot(np.arange(len(z_real)+1)*dt, x_pred[:, 0])
ax[1].set_ylabel('Posición');

## Referencias

- https://www.intechopen.com/chapters/63164
- https://mayitzin.com/2015/06/04/kalman-filter-a-painless-approach/